In [ ]:
import cv2

if not os.path.exists("/content/movie_real.mp4"):
  raise Exception("Video file not found")

if not os.path.exists("/content/vidframes/"):
  os.makedirs("/content/vidframes/")
if not os.path.exists("/content/gendepth/"):
  os.makedirs("/content/gendepth/")

vidcap = cv2.VideoCapture("/content/movie_real.mp4")
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/vidframes/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  count += 1
print("Total frame count: {}".format(count))

In [ ]:
test_model = SkipEncoder6().cuda()

torch.manual_seed(1)

checkpoint_filepath = "/content/drive/My Drive/Checkpoints/demo_model.pth"

if not os.path.exists(checkpoint_filepath):
  raise Exception("Checkpoint file not found. Verify file path?")
      
checkpoint = torch.load(checkpoint_filepath)
test_model.load_state_dict(checkpoint['model_state_dict'])
print("Loaded model from epoch",checkpoint['epoch'],"\nLoss:", checkpoint['loss'])
test_model.eval()

for i in range(count):
  rgb = Image.open("/content/vidframes/frame{}.jpg".format(i))
  rgb = resize(np.array(rgb), (224, 224))
  rgb = transforms.ToTensor()(rgb)
  pred = test_model(rgb.cuda().unsqueeze(0).float()).cpu().detach().numpy()[0].squeeze()
  pred = resize(np.array(pred),(592,480))
  plt.imsave("/content/gendepth/frame{}.jpg".format(i), pred, cmap="inferno")